In [1]:
import pandas as pd
import numpy as np

In [20]:
data_model = pd.read_csv('match_data_bl_wo_nan_s_2022.csv')
data_betting = pd.read_csv('dataframe_with_odd_predictions_v2.csv')
data_betting.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Betting_Prediction,Betting_Prediction_Probability
0,0,0,0,D1,11/08/06,Bayern Munich,Dortmund,2,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,65.423011
1,1,1,1,D1,12/08/06,Hamburg,Bielefeld,1,1,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,71.593915
2,2,2,2,D1,12/08/06,Leverkusen,Aachen,3,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,68.510536
3,3,3,3,D1,12/08/06,Mainz,Bochum,2,1,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,51.671408
4,4,4,4,D1,12/08/06,M'gladbach,Cottbus,2,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,63.117820


In [18]:
data_model_reduced = data_model[['FTR', 'predictedResults', 'ProbAwayWin',
                                'ProbDraw', 'ProbHomeWin']]
data_model_reduced.head()

,FTR,predictedResults,ProbAwayWin,ProbDraw,ProbHomeWin
0,1,0,0.503778,0.210859,0.285363
1,2,2,0.015976,0.036361,0.947663
2,2,0,0.860361,0.039739,0.099900
3,0,2,0.224459,0.236729,0.538812
4,2,2,0.152851,0.198737,0.648412


In [21]:
data_betting_reduced = data_betting[['Betting_Prediction', 'Betting_Prediction_Probability']]


,Betting_Prediction,Betting_Prediction_Probability
0,H,65.423011
1,H,71.593915
2,H,68.510536
3,H,51.671408
4,H,63.117820


In [ ]:
def get_odd_prediction(prob_ges_schnitt):
    erg_odds, erg_prob = [], []
    for x in prob_ges_schnitt:
        max_value = max(x)
        erg_prob.append(max_value)
        if x.index(max_value) == 0:
            erg_odds.append('H')
        if x.index(max_value) == 1:
            erg_odds.append('D')
        if x.index(max_value) == 2:
            erg_odds.append('A')
    return erg_odds, erg_prob